In [ ]:
import pandas as pd 

In [ ]:
population_EHPAD = pd.read_excel(
    'FRA/EHPA_residents.xlsx',
    usecols="A:F",
    skiprows=range(0,5),
    nrows=11,
    sheet_name="T1"
)

In [ ]:
population_EHPAD.rename(columns={ 
    population_EHPAD.columns[0]: "age",
    population_EHPAD.columns[-1]: "Total"
}, inplace = True)
population_EHPAD

In [ ]:
EHPAD_age_map = {
    "60-69": [0, 1],  # Assumes that no one one in the below 65 cat is below 60
    "70-79": [2, 3],
    "80+": [4, 5, 6, 7],
}

In [ ]:
import french_population as fp

In [ ]:
EHPAD_age_table = fp.new_age_table(index=["EHPAD"])
for age in EHPAD_age_map:
    for row in EHPAD_age_map[age]:
        EHPAD_age_table.loc["EHPAD",age] += \
                population_EHPAD.loc[row, "Total"]

EHPAD_age_table["total"] = EHPAD_age_table.sum(axis=1)
EHPAD_age_table["name"] = "EHPAD"
EHPAD_age_table["fra_code"] = "EHPAD"

In [ ]:
EHPAD_age_table

In [ ]:
import extract_opencovidfr_2_ICL as o2i
o2i.data_dir = o2i.Path("./")
o2i.pop_per_region = o2i.read_pop_region(o2i.data_dir + 'FRA/french_population_age_regional.csv')

In [ ]:
fic = "FRA/opencovid19-fr-chiffres-cles.csv"
reg = "REG"
src = o2i.pd.read_csv(fic)

active_regions = o2i.find_active_regions(src, reg)
srcReg = o2i.clean_region_data(src, active_regions)
print(reg + " : " + str(srcReg.shape[0]) + " lignes")

dst = o2i.convert_opencovidfr_to_ICL_model(srcReg, o2i.pop_per_region)




In [ ]:
def calculate_daily_change(df, region_id, cumulated_field, field):
    reg_logicind = df["geoId"] == region_id
    reg_deaths = np.array(df.loc[reg_logicind, cumulated_field])
    for i, deaths in enumerate(reg_deaths):
        if i>0 and deaths < reg_deaths[i-1]:
            reg_deaths[i] = reg_deaths[i-1]
    reg_deaths = reg_deaths-[0, *reg_deaths[:-1]]
    df.loc[reg_logicind, field] = reg_deaths

In [ ]:
dst.sort_values("t", inplace=True)
active_regions = dst["geoId"].unique()
dst["deaths"] = 0
for region in active_regions:
    calculate_daily_change(dst, region, "cumulated_deaths", "deaths")
    calculate_daily_change(dst, region, "cumulated_cases", "cases")

In [ ]:
import numpy as np

In [ ]:
reg_logicind = dst["geoId"] == "REG-84"
reg_deaths = np.array(dst.loc[reg_logicind, "cumulated_deaths"])
for i, deaths in enumerate(reg_deaths):
    if i>0 and deaths < reg_deaths[i-1]:
        reg_deaths[i] = reg_deaths[i-1]
reg_deaths = reg_deaths-[0, *reg_deaths[:-1]]
dst.loc[reg_logicind, "deaths"] = reg_deaths

In [ ]:
dst